In [1]:
import json
import codecs
import uuid
import ast
import requests
import google.cloud.dialogflow_v2 as dialogflow
import time

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key_service.json"

In [2]:
rasaStoriesFile = 'converted_files/data/stories.md'
rasaUtterancesFile = 'converted_files/data/nlu.md'
rasaDomainFile = 'converted_files/domain.yml'

flowEntities = 'dialogflow/entities/'

PROJECT_ID = 'test2-iovj'
DEVELOPER_ACCESS_TOKEN = 'HIDDEN'

In [3]:
entities = []
entity_set = {}

In [4]:
def get_entities_from_domain(domain):
    
    printer = False
    first_line = True
    entities = []
    
    domain_options = ['slots:\n', 'actions:\n', 'responses:\n', 'intents:\n']
    
    with codecs.open(domain, 'r', 'utf-8') as f_out:
        for line in f_out:
            
            first_line = True
            
            if line == 'entities:\n':
                printer = True
                first_line = False
                
            if line in domain_options:
                printer = False
                
            if printer and first_line:
                line = line.strip()
                line = line.replace(' ', '')
                line = line.replace('-', '')
                entities.append(line)

    return entities[:-1]

entities = get_entities_from_domain(rasaDomainFile)

### create entity set

In [5]:
for entity in entities:
#     description = {
#       "id": str(uuid.uuid4()),
#       "name": entity,
#       "isOverridable": True,
#       "isEnum": False,
#       "isRegexp": False,
#       "automatedExpansion": True,
#       "allowFuzzyExtraction": True
#     }    
    
#     with open(flowEntities + entity +'.json', 'w') as outfile:
#         json.dump(description, outfile,indent=2)
        
    entity_set[entity] = set()


### Get entity examples

In [6]:
def get_entity_examples_from_stories(stories):
    
    entity_set['type'] = set()
    with codecs.open(stories, 'r', 'utf-8') as f_out:
        for line in f_out:
            for entity in entities:
                if entity in line:

                    myLine = line[line.index(entity):]
                    if ':' in myLine:
                        myLine = myLine[myLine.index(':'):]
                        myLine = myLine[3:]
                        val = myLine[:myLine.index('"')]
                        
                        entity_set[entity] = entity_set[entity] |{val} 
                                    
get_entity_examples_from_stories(rasaStoriesFile)

In [7]:
def get_entity_examples_from_nlu(nlu):

    entity_set['type'] = set()
    with codecs.open(nlu, 'r', 'utf-8') as f_out:
        for line in f_out:
            for entity in entities:
                padded_ent = '('+entity+')'
                if padded_ent in line:

                    myLine = line[:line.index(entity)]
                    val = myLine[myLine.index('['):][1:-2]
                        
                    entity_set[entity] = entity_set[entity] |{val} 
                        
get_entity_examples_from_nlu(rasaUtterancesFile)

## API Calls

In [8]:
def create_entity(project_id, entity_type_id, entity_value, synonyms):
    """Create an entity of the given entity type."""
    entity_types_client = dialogflow.EntityTypesClient()

    # Note: synonyms must be exactly [area] if the
    # entity_type's kind is KIND_LIST
    synonyms = synonyms or [entity_value]

    entity_type_path = entity_types_client.entity_type_path(
        project_id, entity_type_id)

    entity = dialogflow.types.EntityType.Entity()
    entity.value = entity_value
    entity.synonyms.extend(synonyms)

    response = entity_types_client.batch_create_entities({
        'parent': entity_type_path, 'entities':[entity]})

    print('Entity created: {}'.format(response))

In [9]:
def create_entity_type(project_id, display_name):
    """Create an entity type with the given display name."""
    
    kind = dialogflow.types.EntityType.Kind.KIND_MAP
    
    entity_type = dialogflow.types.EntityType(
        display_name=display_name, kind=kind)
    
    entity_types_client = dialogflow.EntityTypesClient();
 
    parent = dialogflow.AgentsClient.agent_path(project_id)

    response = entity_types_client.create_entity_type({'parent':parent, 'entity_type': entity_type})

    
    print('Entity type created: \n{}'.format(response))
    
    return response

## send to API

In [11]:
counter = 0

for ent_set in entity_set:
    
    
    entity_out = create_entity_type(PROJECT_ID, ent_set)
    entity_id = entity_out.name[-36:]
        
    for e in entity_set[ent_set]:
        
        if e == '':
            continue

        create_entity(PROJECT_ID, entity_id,e, [e])
        counter +=1 
        
        if not counter % 20:
            print('waiting...')
            time.sleep(20)
            print('wait over')

Entity type created: 
name: "projects/test2-iovj/agent/entityTypes/8be5b44d-578d-4ffd-ae04-393236acf17e"
display_name: "area"
kind: KIND_MAP

Entity created: <google.api_core.operation.Operation object at 0x10da74ac0>
Entity created: <google.api_core.operation.Operation object at 0x10da74340>
Entity created: <google.api_core.operation.Operation object at 0x10da74730>
Entity created: <google.api_core.operation.Operation object at 0x10b68c520>
Entity created: <google.api_core.operation.Operation object at 0x10da74730>
Entity created: <google.api_core.operation.Operation object at 0x10da802e0>
Entity type created: 
name: "projects/test2-iovj/agent/entityTypes/82f7cb19-c7cd-464f-b65b-780e9ce345b3"
display_name: "pricerange"
kind: KIND_MAP

Entity created: <google.api_core.operation.Operation object at 0x10da80340>
Entity created: <google.api_core.operation.Operation object at 0x10da800a0>
Entity created: <google.api_core.operation.Operation object at 0x10da80520>
Entity created: <google.ap

Entity created: <google.api_core.operation.Operation object at 0x10da88130>
Entity created: <google.api_core.operation.Operation object at 0x10da88580>
Entity created: <google.api_core.operation.Operation object at 0x10da88a60>
Entity created: <google.api_core.operation.Operation object at 0x10da884f0>
waiting...
wait over
Entity type created: 
name: "projects/test2-iovj/agent/entityTypes/a68e4965-2ad8-4c5a-8069-83879f53711b"
display_name: "type"
kind: KIND_MAP

